In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [19]:
# Read in and look at the data
df = pd.read_csv('stock_trans_data.csv')

df.head()

,symbol,historical_dates,open,close,purchase_date,purchase_price,sell_date,sell_price,actual_return,days_to_sell,take_profit_price,stop_out_price,hit_take_profit
0,CRM,"[datetime.date(2018, 12, 3), datetime.date(201...","[146.3, 142.73, 134.8, 141.9, 135.16, 140.52, ...","[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...",2023-11-20,223.20,2023-11-30,249.48,315.37,10.0,231.57,220.9680,1
1,NET,"[datetime.date(2019, 9, 13), datetime.date(201...","[18.1, 18.6, 18.45, 18.83, 20, 18.9, 20.1, 21....","[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...",2023-11-20,73.37,2023-12-14,83.38,340.34,24.0,81.62,72.6363,1
2,RCL,"[datetime.date(2018, 11, 23), datetime.date(20...","[107.1, 111.04, 110.53, 111.38, 113.05, 111.2,...","[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...",2023-11-20,105.26,2023-11-21,102.96,-57.50,1.0,107.73,104.2074,0
3,SPOT,"[datetime.date(2018, 12, 6), datetime.date(201...","[131.01, 136.89, 134.2, 133.81, 130.3, 129.78,...","[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...",2023-11-20,178.86,2023-12-04,193.85,224.84,14.0,187.08,177.0714,1
4,FERG,"[datetime.date(2021, 3, 8), datetime.date(2021...","[120.51, 123, 124, 124, 123.72, 125.48, 126.66...","[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...",2023-11-20,164.78,2023-11-20,164.63,-2.41,0.0,165.06,163.1322,0


In [20]:
# Function to calculate SMA5
def calculate_sma5(close_prices):
    sma5 = []
    for i in range(len(close_prices)):
        if i < 4:  # Predict the SMA5 for the first four dates
            sma5.append(close_prices[i])
        else:
            sma5.append(np.mean(close_prices[i-4:i+1]))
    return sma5

# Apply the function to the 'close' column
df['SMA5'] = df['close'].apply(lambda x: calculate_sma5(eval(x)))

# Inspect the result
df[['close', 'SMA5']].head()

,close,SMA5
0,"[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...","[144.15, 139.24, 142.06, 135.95, 139.733999999..."
1,"[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...","[18, 18.63, 18.75, 19.59, 18.744, 19.118000000..."
2,"[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...","[110.28, 111.46, 110.74, 112.54, 111.330000000..."
3,"[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...","[136.83, 135.31, 130.79, 128.37, 131.954, 129...."
4,"[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...","[119.49, 122.35, 120.6, 125, 122.4159999999999..."


In [21]:
def calculate_sma20(close_prices):
    sma20 = []
    for i in range(len(close_prices)):
        if i < 19:  # Not enough data for SMA5
            sma20.append(close_prices[i])
        else:
            sma20.append(np.mean(close_prices[i-19:i+1]))
    return sma20

# Apply the function to the 'close' column
df['SMA20'] = df['close'].apply(lambda x: calculate_sma20(eval(x)))

# Inspect the result
df[['close', 'SMA20']].head()

,close,SMA20
0,"[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...","[144.15, 139.24, 142.06, 135.95, 137.27, 136.7..."
1,"[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...","[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,..."
2,"[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...","[110.28, 111.46, 110.74, 112.54, 111.63, 113.0..."
3,"[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...","[136.83, 135.31, 130.79, 128.37, 128.47, 125.5..."
4,"[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...","[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1..."


In [22]:
# Function to calculate SMA5 slope based on the previous 4 days
def calculate_sma5_slope_with_prediction(sma5_list):
    sma5_slope = []
    for i in range(len(sma5_list)):
        if i < 4:  # Predict the slope for the first four entries
            if i > 0:  # Calculate the slope between the first two available points
                predicted_slope = round((sma5_list[i] - sma5_list[i-1]) , 2)
            else:
                predicted_slope = 0  # If no prior data, assume flat slope
            sma5_slope.append(predicted_slope)
        else:
            # Slope = (SMA5[today] - SMA5[4 days ago]) / 4
            slope = round((sma5_list[i] - sma5_list[i-4]) / 4, 2)
            sma5_slope.append(slope)
    return sma5_slope

# Apply the function to the 'close' column
df['SMA5_Slope'] = df['SMA5'].apply(lambda x: calculate_sma5_slope_with_prediction(x))

# Inspect the result
df[['SMA5', 'SMA5_Slope']].head()

,SMA5,SMA5_Slope
0,"[144.15, 139.24, 142.06, 135.95, 139.733999999...","[0, -4.91, 2.82, -6.11, -1.1, -0.24, -0.91, 0...."
1,"[18, 18.63, 18.75, 19.59, 18.744, 19.118000000...","[0, 0.63, 0.12, 0.84, 0.19, 0.12, 0.21, 0.1, 0..."
2,"[110.28, 111.46, 110.74, 112.54, 111.330000000...","[0, 1.18, -0.72, 1.8, 0.26, 0.11, 0.43, -0.1, ..."
3,"[136.83, 135.31, 130.79, 128.37, 131.954, 129....","[0, -1.52, -4.52, -2.42, -1.22, -1.4, -0.75, -..."
4,"[119.49, 122.35, 120.6, 125, 122.4159999999999...","[0, 2.86, -1.75, 4.4, 0.73, 0.38, 1.01, 0.1, 0..."


In [23]:
# Function to calculate SMA5 slope based on the previous 4 days
def calculate_sma20_slope_with_prediction(sma20_list):
    sma20_slope = []
    for i in range(len(sma20_list)):
        if i < 19:  # Predict the slope for the first four entries
            if i > 0:  # Calculate the slope between the first two available points
                predicted_slope = round((sma20_list[i] - sma20_list[i-1]) , 2)
            else:
                predicted_slope = 0  # If no prior data, assume flat slope
            sma20_slope.append(predicted_slope)
        else:
            slope = round((sma20_list[i] - sma20_list[i-4]) / 19, 2)
            sma20_slope.append(slope)
    return sma20_slope

# Apply the function to the 'close' column
df['SMA20_Slope'] = df['SMA20'].apply(lambda x: calculate_sma20_slope_with_prediction(x))

# Inspect the result
df[['SMA20', 'SMA20_Slope']].head()

,SMA20,SMA20_Slope
0,"[144.15, 139.24, 142.06, 135.95, 137.27, 136.7...","[0, -4.91, 2.82, -6.11, 1.32, -0.48, 3.3, 1.05..."
1,"[18, 18.63, 18.75, 19.59, 18.75, 19.87, 20.96,...","[0, 0.63, 0.12, 0.84, -0.84, 1.12, 1.09, -0.25..."
2,"[110.28, 111.46, 110.74, 112.54, 111.63, 113.0...","[0, 1.18, -0.72, 1.8, -0.91, 1.44, 1.18, -5.12..."
3,"[136.83, 135.31, 130.79, 128.37, 128.47, 125.5...","[0, -1.52, -4.52, -2.42, 0.1, -2.92, 0.13, -5...."
4,"[119.49, 122.35, 120.6, 125, 124.64, 126.66, 1...","[0, 2.86, -1.75, 4.4, -0.36, 2.02, -0.33, -1.8..."


In [24]:
from statistics import mode

# Create a new list of 1s and 0s. 1 = increase from one sma to the next days sma, 0 = decrease from on sma to the next days sma
def calculate_increases(lst):
    probabilities = []
    for i in range(len(lst) - 1):
        if lst[i] < lst[i + 1]:
            probabilities.append(1)
        else:
            probabilities.append(0)
    probabilities.append(0)
    most_common = mode(probabilities)
    return most_common

# create new SMA<value>_prob columns and SMA<value>_slope_prob columns based on the result of the calculate increase function.
def get_increase_probability_and_create_new_column(initial_column, created_column):
    probs = []
    for x in df[initial_column]:
        prob = calculate_increases(x)
        probs.append(prob)
    df[created_column] = probs

# create a list of column names to run this on
old_columns = ['SMA5', 'SMA20', 'SMA5_Slope', 'SMA20_Slope']
for name in old_columns:
    get_increase_probability_and_create_new_column(name, name+'_prob')
    
# Calculate summary statistics for 'open' and 'close' prices
df['open_mean'] = df['open'].apply(lambda x: np.mean(eval(x)))
df['open_std'] = df['open'].apply(lambda x: np.std(eval(x)))
df['close_mean'] = df['close'].apply(lambda x: np.mean(eval(x)))
df['close_std'] = df['close'].apply(lambda x: np.std(eval(x)))
df['SMA5_last'] = df['SMA5'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)
df['SMA20_last'] = df['SMA20'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)
df['SMA5_Slope_last'] = df['SMA5_Slope'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)
df['SMA20_Slope_last'] = df['SMA20_Slope'].apply(lambda x: x[-1] if isinstance(x, list) else np.nan)


# drop all columns that contains lists as their data
df = df.drop(['SMA5','SMA20','SMA5_Slope','SMA20_Slope','open','close','historical_dates', 'actual_return'], axis=1)
# inspect the dataframe in its current state
df

,symbol,purchase_date,purchase_price,sell_date,sell_price,days_to_sell,take_profit_price,stop_out_price,hit_take_profit,SMA5_prob,...,SMA5_Slope_prob,SMA20_Slope_prob,open_mean,open_std,close_mean,close_std,SMA5_last,SMA20_last,SMA5_Slope_last,SMA20_Slope_last
0,CRM,2023-11-20,223.20,2023-11-30,249.48,10.0,231.57,220.9680,1,1,...,0,0,194.874448,41.238154,194.878719,41.079058,231.268,219.90650,1.87,0.35
1,NET,2023-11-20,73.37,2023-12-14,83.38,24.0,81.62,72.6363,1,1,...,0,0,66.832950,38.665226,66.932882,38.640908,79.714,76.48300,0.49,0.14
2,RCL,2023-11-20,105.26,2023-11-21,102.96,1.0,107.73,104.2074,0,1,...,0,0,81.513144,25.881074,81.473604,25.901222,104.394,93.66850,1.59,0.25
3,SPOT,2023-11-20,178.86,2023-12-04,193.85,14.0,187.08,177.0714,1,1,...,1,0,174.833222,67.292948,174.899467,67.200348,184.978,177.20050,0.88,0.19
4,FERG,2023-11-20,164.78,2023-11-20,164.63,0.0,165.06,163.1322,0,1,...,1,0,138.396361,17.321011,138.696428,17.230391,165.182,157.66450,1.37,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,REG,2024-08-07,70.38,NaN,NaN,NaN,72.46,69.6762,0,1,...,0,0,59.917562,9.372880,59.899741,9.399800,70.448,69.32250,0.13,0.05
296,WELL,2024-08-12,115.96,NaN,NaN,NaN,119.41,114.8004,0,1,...,1,0,78.236523,14.707191,78.213986,14.713709,116.408,114.68500,-0.08,0.08
297,K,2024-08-12,74.29,2024-08-14,80.09,2.0,76.50,73.5471,1,1,...,0,0,64.184265,5.475001,64.174996,5.475372,75.600,64.91800,1.06,0.20
298,VIRT,2024-08-15,30.31,NaN,NaN,NaN,31.17,30.0069,0,1,...,0,0,23.079552,4.875443,23.069714,4.871222,29.718,28.61500,0.07,0.01


In [25]:
from sklearn.preprocessing import LabelEncoder
# Initialize the Label Encoder
label_encoder = LabelEncoder()
# Fit and transform the 'symbol' column
df['symbol_encoded'] = label_encoder.fit_transform(df['symbol'])
# Fit and transform the 'purchase_date' column
df['purchase_date_encoded'] = label_encoder.fit_transform(df['purchase_date'])
# Fit and transform the 'sell_date' column
df['sell_date_encoded'] = label_encoder.fit_transform(df['sell_date'])

# inspect the dataframe in its current state
df


,symbol,purchase_date,purchase_price,sell_date,sell_price,days_to_sell,take_profit_price,stop_out_price,hit_take_profit,SMA5_prob,...,open_std,close_mean,close_std,SMA5_last,SMA20_last,SMA5_Slope_last,SMA20_Slope_last,symbol_encoded,purchase_date_encoded,sell_date_encoded
0,CRM,2023-11-20,223.20,2023-11-30,249.48,10.0,231.57,220.9680,1,1,...,41.238154,194.878719,41.079058,231.268,219.90650,1.87,0.35,49,0,7
1,NET,2023-11-20,73.37,2023-12-14,83.38,24.0,81.62,72.6363,1,1,...,38.665226,66.932882,38.640908,79.714,76.48300,0.49,0.14,105,0,16
2,RCL,2023-11-20,105.26,2023-11-21,102.96,1.0,107.73,104.2074,0,1,...,25.881074,81.473604,25.901222,104.394,93.66850,1.59,0.25,128,0,1
3,SPOT,2023-11-20,178.86,2023-12-04,193.85,14.0,187.08,177.0714,1,1,...,67.292948,174.899467,67.200348,184.978,177.20050,0.88,0.19,147,0,9
4,FERG,2023-11-20,164.78,2023-11-20,164.63,0.0,165.06,163.1322,0,1,...,17.321011,138.696428,17.230391,165.182,157.66450,1.37,0.11,68,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,REG,2024-08-07,70.38,NaN,NaN,NaN,72.46,69.6762,0,1,...,9.372880,59.899741,9.399800,70.448,69.32250,0.13,0.05,129,87,106
296,WELL,2024-08-12,115.96,NaN,NaN,NaN,119.41,114.8004,0,1,...,14.707191,78.213986,14.713709,116.408,114.68500,-0.08,0.08,178,88,106
297,K,2024-08-12,74.29,2024-08-14,80.09,2.0,76.50,73.5471,1,1,...,5.475001,64.174996,5.475372,75.600,64.91800,1.06,0.20,93,88,104
298,VIRT,2024-08-15,30.31,NaN,NaN,NaN,31.17,30.0069,0,1,...,4.875443,23.069714,4.871222,29.718,28.61500,0.07,0.01,169,89,106


In [26]:
# Create a seperate dataframe for the rows that are still showing as the position being open. This will be used for prediction
future_df = df[(df['sell_price'].isna())]
future_df = future_df.copy()

# drop the open positions from the training and test data
df = df.dropna(subset=['sell_price'])


# Now you have the last 5 open and close prices as features for your model create model used for testing future data
X = df.drop(['hit_take_profit', 'symbol', 'purchase_date', 'sell_date'], axis=1)
y = df['hit_take_profit']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)



Accuracy: 0.7457627118644068
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.57      0.63        23
           1       0.76      0.86      0.81        36

    accuracy                           0.75        59
   macro avg       0.74      0.71      0.72        59
weighted avg       0.74      0.75      0.74        59



In [27]:
X_future = future_df.drop(['hit_take_profit', 'symbol', 'purchase_date', 'sell_date'], axis=1)
future_predictions = rf_model.predict(X_future)

In [28]:
future_df['hit_take_profit_predicted'] = future_predictions

In [29]:
future_df.to_csv('future_predictions.csv', index=False)

In [30]:
future_probabilities = rf_model.predict_proba(X_future)
future_probabilities

array([[0.39, 0.61],
       [0.39, 0.61],
       [0.32, 0.68],
       [0.79, 0.21],
       [0.7 , 0.3 ]])

In [31]:
print(df['hit_take_profit'].value_counts())


hit_take_profit
1    181
0    114
Name: count, dtype: int64


In [32]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rf_model, X, y, cv=5)
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", np.mean(cv_scores))

Cross-validation scores: [0.6779661  0.69491525 0.79661017 0.83050847 0.71186441]
Mean cross-validation score: 0.7423728813559322


In [33]:
importances = rf_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
print(feature_importance_df.sort_values(by='importance', ascending=False))


                  feature  importance
15        SMA5_Slope_last    0.182009
2            days_to_sell    0.106461
16       SMA20_Slope_last    0.096195
19      sell_date_encoded    0.064787
17         symbol_encoded    0.057362
1              sell_price    0.051763
18  purchase_date_encoded    0.049884
9               open_mean    0.048955
12              close_std    0.047411
14             SMA20_last    0.045051
11             close_mean    0.043584
3       take_profit_price    0.041999
0          purchase_price    0.041993
10               open_std    0.040308
4          stop_out_price    0.035743
13              SMA5_last    0.035705
7         SMA5_Slope_prob    0.004326
5               SMA5_prob    0.003941
6              SMA20_prob    0.002524
8        SMA20_Slope_prob    0.000000
